#16. Transform Orders data - String to JSON
1. pre-process JSON string to fix data quality issues.
2. transform JSON string to JSON object.
3. Write transformed data to silver schema.

###1. pre-process JSON string to fix data quality issues

In [0]:
%sql
SELECT value,
       regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})','"order_date": "\$1"') AS fixed_value
FROM shoppix.bronze.v_orders

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_fixed
AS 
SELECT value,
       regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})','"order_date": "\$1"') AS fixed_value
FROM shoppix.bronze.v_orders

###2. Transform JSON string to JSON object
    * function schema_of_json 
    * function from_json 

In [0]:
%sql
SELECT schema_of_json(fixed_value) AS schema,
       fixed_value
FROM tv_orders_fixed
LIMIT 1;

In [0]:
%sql
SELECT from_json(fixed_value,
                'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
                fixed_value
FROM tv_orders_fixed;


###3. Write transformed data to silver schema

In [0]:
%sql
CREATE TABLE IF NOT EXISTS shoppix.silver.orders_json
AS
SELECT from_json(fixed_value,
                'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
                fixed_value
FROM tv_orders_fixed;